In [42]:
from lxml import etree
import numpy as np

In [57]:
tree = etree.parse('./minimal_sample.xml')

In [58]:
for child in tree.getroot().getchildren():
    print(child.tag, child.getchildren())

PRODUCT []
COLLECTION [<Element TRACK at 0x1157530c0>]


In [76]:
tracks = tree.getroot().xpath('//COLLECTION/TRACK')

for track in tracks:
    track_name = track.attrib['Name']
    bpms = track.xpath('TEMPO/@Bpm')
    bpms = [float(i) for i in bpms]

    if len(bpms) == 0:
        print(f"{track_name} not analyzed, skipping")
        continue
    if max(bpms) != np.mean(bpms):
        print(f'Warning - Track has multiple BPMs: {track_name} ')
    bpm = bpms[0]

    if "PROCESSED" in track.xpath('POSITION_MARK/@Name'):
        print("Track already processed, skipping")
        continue

    cues = track.xpath('POSITION_MARK')
    print(cues)
    if len(cues) > 1:
        print(f"SKIPPING - Unprocessed track has multiple cue points: {track_name}")
        continue
    if len(cues) == 0:
        print(f"SKIPPING - Unprocessed track has no cue points: {track_name}")
        continue

    #ASSUMES FIRST CUE IS AT THE DROP
    
    #get cue timestamp
    drop_timestamp = float(cues[0].attrib['Start'])
    #create new cue points 8/16/24/32 bars before
    intervals = [8,16,24,32]
    beats = [x*4 for x in intervals]
    seconds_per_beat = 60/bpm

    new_cue_timestamps = [drop_timestamp - (x * seconds_per_beat) for x in beats]
    for ts in new_cue_timestamps:
        if ts > 0:
            ts = format(ts, '.3f')
            new_cue = etree.Element("POSITION_MARK", Name="PROCESSED", Type="0", Start=ts, Num="-1")
            track.insert(1, new_cue)
    print(new_cue_timestamps)
    #make sure time >0 on all new cues

[<Element POSITION_MARK at 0x115753c00>]
[44.82599999999999, 32.025999999999996, 19.225999999999992, 6.425999999999995]


In [79]:
tree.write('output.xml', pretty_print=True, encoding="utf-8", xml_declaration=True)